# TESTE A/B 

# Dicionário dos Dados

|Atributo|Tipo|Descrição|
|-------|----|-----------|
|uid|int64| Identificador único do cliente|
|country|object| País do cliente|
|gender|object| Gênero do cliente|
|spent|int64| Total gasto na transação|
|purchases|int64| Número de compras realizadas na transação|
|date|datetime64[ns]| Data da compra|
|group|object| GROUP B (Controle) ->  Manual , GROUP A (Tratamento) -> Automático|
|device |object| Dispositivo em que a compra foi realizada (I-> Site, A -> App)|

# 1.0 Imports

In [1]:
import pandas as pd
import numpy as np
from statsmodels.stats import api as sms
import statsmodels.api as sma
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as st
from src import GeneralUtils as gu
import plotly.express as px
import warnings
from cliffs_delta import cliffs_delta

In [2]:
test_lift = lambda mean2, mean1: (mean2 - mean1)/mean1

In [3]:
warnings.filterwarnings('ignore')
sns.set_style('darkgrid')
pd.set_option('display.max_colwidth', None)

## 1.1 Load Data

In [4]:
df_raw = pd.read_csv('../data/ab_testing.csv')
df_raw.head()

,uid,country,gender,spent,purchases,date,group,device
0,11115722,MEX,F,1595,5,2016-03-08,GRP B,I
1,11122053,USA,M,498,2,2017-07-14,GRP B,I
2,11128688,USA,F,2394,6,2017-09-17,GRP A,I
3,11130578,USA,F,1197,3,2017-11-30,GRP A,I
4,11130759,ESP,M,1297,3,2018-01-10,GRP B,A


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)


# 2.0 Descrição dos Dados

In [5]:
df1 = df_raw.copy()

## 2.2 Checando valores faltantes e Tipos de dados

In [6]:
print(f'Number of rows: {df1.shape[0]}')
print(f'Number of Cols: {df1.shape[1]}')

Number of rows: 45883
Number of Cols: 8


## 2.2 Checando valores faltantes e Tipos de dados

In [7]:
gu.sum_table(df1)

,Name,dtypes,Uniques,Missing,Missing %
0,uid,int64,45883,0,0.0
1,country,object,10,0,0.0
2,gender,object,2,0,0.0
3,spent,int64,641,0,0.0
4,purchases,int64,26,0,0.0
5,date,object,1316,0,0.0
6,group,object,2,0,0.0
7,device,object,2,0,0.0


## 2.3. Corrigindo Tipos de Dados

In [8]:
df1['date'] = pd.to_datetime(df1['date'])
df_aux = df1.copy()
df_aux['year_month'] = df_aux['date'].dt.strftime('%Y-%m')

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)


# 3.0 Design de Experimento

Para realizar este experimento, nós iremos realizar testes para cada país por tipo de dispositivo.

O time de negócio definiu que, para valer a pena implementar esta mudança, é necessário um aumento de 8% na média de gastos.

## 3.1. Site
Experimento: Mudança na forma de pagamento da página de vendas.

Hipótese Nula: O preenchimento automático e o preenchimento manual possuem médias de gastos iguais.

Hipótese Alternativa: As médias de gastos das duas formas de pagamento são diferentes.

Unidade de Randomização -> Usuários.

População da unidade de randomização -> Usuários que realizaram compras no site.


## 3.2. App
Experimento: Mudança na forma de pagamento da página de vendas.

Hipótese Nula: O preenchimento automático e o preenchimento manual possuem médias de gastos iguais.

Hipótese Alternativa: As médias de gastos das duas formas de pagamento são diferentes.

Unidade de Randomização -> Usuários.

População da unidade de randomização -> Usuários que realizaram compras no app.



## 3.3 Parâmentros (definição do tamanho das amostras).

Para este experimento, os seguintes parâmetros:

nível de significância = 0.05

poder estatístico = 0.80

efeito prático ->

O efeito prático pode ser definido utilizando dados históricos, mas neste dataset há apenas os dados do experimento, então iremos calcular o effect size com base nos dados do grupo controle de cada país e tipo de dispositivos.

Outra maneira seria definir um valor arbitrário para o cohen's d e usá-lo para o cálculo do tamanho da amostra necessário. Mas esta prática não é recomendada, pois não temos uma noção muito boa do tamanho do aumento — cohen's d é adimensional.

effect_size = avg_control * (1 + lift) -  avg_control

avg_control = média do grupo controle.
lift = ganho esperado pelo time de negócio.

In [9]:
confidence_lvl = 0.95

significance_lvl = 0.05

lift = 0.08
#effect_size = avg_control * (1 + lift) -  avg_control
power = 0.80

In [10]:
import warnings
warnings.filterwarnings('ignore')

## 3.4. Verificando se há amostras suficientes para o teste em cada país.

In [11]:
countries = df1.country.unique()
dataframes = {}
for country in countries:

    country_df = df1.query(f"country == '{country}'")
    dataframes[country] = country_df

In [12]:
sizes_df = pd.DataFrame()
data_sizes_countries_site = {}

for country in countries:
    
    dfs = gu.data(dataframes[country])
    x_b = dfs['site_b'].spent
    stdev = x_b.std()
    avg1 = x_b.mean()
    effect_size = (avg1 * (1 + lift) - avg1)/stdev

    size = gu.sample_n(alpha=significance_lvl, power=power, effect_size=effect_size)
    data_sizes_countries_site[country] = size

    size_site_a = len(dfs['site_a'])
    size_site_b = len(dfs['site_b'])

    status = "Not enough data to run the test." if min(size_site_a, size_site_b) < size else "There is enough data to run the test."

    result = {'Country': country, 'Device': 'site', 'Sample size': size, 'Group B': size_site_b, 'Group A': size_site_a, 'Status': status}

    sizes_df = pd.concat([sizes_df, pd.Series(result)], axis=1)

sizes_df.T.reset_index(drop=True)

,Country,Device,Sample size,Group B,Group A,Status
0,MEX,site,1242,1355,1435,There is enough data to run the test.
1,USA,site,1277,3531,3426,There is enough data to run the test.
2,ESP,site,1168,473,521,Not enough data to run the test.
3,GBR,site,1299,691,684,Not enough data to run the test.
4,TUR,site,1281,880,935,Not enough data to run the test.
5,DEU,site,1325,917,931,Not enough data to run the test.
6,BRA,site,1310,2207,2310,There is enough data to run the test.
7,FRA,site,1313,694,680,Not enough data to run the test.
8,AUS,site,1257,259,239,Not enough data to run the test.
9,CAN,site,1238,372,361,Not enough data to run the test.


In [13]:
sizes_df_app = pd.DataFrame()
data_sizes_countries_app = {}

for country in countries:
    
    dfs = gu.data(dataframes[country])
    x_b = dfs['app_b'].spent
    stdev = x_b.std()
    avg1 = x_b.mean()
    effect_size = (avg1 * (1 + lift) - avg1)/stdev
    
    size = gu.sample_n(alpha=significance_lvl, power=power, effect_size=effect_size)
    data_sizes_countries_app[country] = size
    
    size_app_a = len(dfs['app_a'])
    size_app_b = len(dfs['app_b'])

    status = "Not enough data to run the test." if min(size_app_a, size_app_b) < size else "There is enough data to run the test."

    result = {'Country': country, 'Device': 'site', 'Sample size': size, 'Group B': size_app_b, 'Group A': size_app_a, 'Status': status}

    sizes_df_app = pd.concat([sizes_df_app, pd.Series(result)], axis=1)

sizes_df_app.T.reset_index(drop=True)

,Country,Device,Sample size,Group B,Group A,Status
0,MEX,site,1275,1287,1451,There is enough data to run the test.
1,USA,site,1291,3540,3501,There is enough data to run the test.
2,ESP,site,1115,491,449,Not enough data to run the test.
3,GBR,site,1221,732,702,Not enough data to run the test.
4,TUR,site,1381,862,860,Not enough data to run the test.
5,DEU,site,1150,889,959,Not enough data to run the test.
6,BRA,site,1281,2284,2208,There is enough data to run the test.
7,FRA,site,1290,715,753,Not enough data to run the test.
8,AUS,site,1405,252,267,Not enough data to run the test.
9,CAN,site,1175,443,337,Not enough data to run the test.


Só temos dados suficientes, com os parâmetros definidos, para realizar o teste para México, Estados Unidos e Brasil.

## 3.5 Homegeneidade das amostras

In [14]:
df1.query("country in ('MEX', 'USA', 'BRA')").groupby(['country',  'group', 'gender', 'device']).count()[['uid']].sort_values(['country', 'device', 'gender'])


uid
country group gender device      
BRA     GRP A F      A       1044
        GRP B F      A       1135
        GRP A M      A       1164
        GRP B M      A       1149
        GRP A F      I       1169
        GRP B F      I       1073
        GRP A M      I       1141
        GRP B M      I       1134
MEX     GRP A F      A        692
        GRP B F      A        630
        GRP A M      A        759
        GRP B M      A        657
        GRP A F      I        712
        GRP B F      I        688
        GRP A M      I        723
        GRP B M      I        667
USA     GRP A F      A       1779
        GRP B F      A       1762
        GRP A M      A       1722
        GRP B M      A       1778
        GRP A F      I       1662
        GRP B F      I       1715
        GRP A M      I       1764
        GRP B M      I       1816

The proportion of users by gender in each group is similar.

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)


# 4.0. Testes

Para realizar os testes iremos separar os dados por país e por dispositivos. Faremos isso pois o comportamento dos clientes pode ser muito diferente para essas duas variáveis.

Como foi visto na seção anterior, os teste serão feitos apenas para Estados Unidos, Brasil e México.

## 4.1. Premissas do teste paramétrico.

Para cada teste iremos vericar os seguintes pressupostos.
- Amostras independentes.
- A distribuição das médias amostrais é normal.
- As variâncias dos dois grupos são iguais.

O atendidemento ou não desses pressupostos irá definir qual teste de hipótese utilizaremos.


|Distribution|Variance|Test
|-----|------|------|
Normally Distributed| Equal Variance| Student's t-Test
Normally Distributed| Different Variance| Welch's t-Test
Not Normally Distributed|| Mann Whitney U test


Para verificar se as amostras são normalmente distribuídas, utilizaremos o teste de Shapiro-Wilk com um nível de significância de 0.05.

Para verrificar se as variâncias são iguais, utilizaremos o test de Levene, também com um nível de significância de 0.05.

## 4.2. Hipóteses

**Hipótese Nula**: O preenchimento automático e o preenchimento manual possuem médias de gastos iguais.

**Hipótese Alternativa**: O preenchimento automático e o preenchimento manual possuem médias de gastos diferentes.

## México

In [15]:
dfs = gu.data(dataframes['MEX'])

### Site

In [16]:
sample_n = data_sizes_countries_site['MEX']

x_site_a = dfs['site_a'].sample(sample_n, random_state=0).spent
x_site_b = dfs['site_b'].sample(sample_n, random_state=0).spent

print(f'Site - Grupo A -> gasto médio: {np.mean(x_site_a):.2f}')
print(f'Site - Grupo B -> gasto médio: {np.mean(x_site_b):.2f}\n')
gu.test_results(x_site_a, x_site_b, alpha=significance_lvl, verbose=True)

print(f'Test Lift: {test_lift(x_site_a.mean(), x_site_b.mean()):.2%}')

Site - Grupo A -> gasto médio: 1922.62
Site - Grupo B -> gasto médio: 1846.11

Mann Whitney U Test

P-value: 0.1204
Falha em rejeitar a hipótese nula.
Não há evidências suficientes no nível de significância 0.05 para dizer que existe diferença entre as médias  dos grupos.
Test Lift: 4.14%


### App

In [17]:
sample_n = data_sizes_countries_app['MEX']
x_app_a = dfs['app_a'].sample(sample_n, random_state=0).spent
x_app_b = dfs['app_b'].sample(sample_n, random_state=0).spent

print(f'App - Grupo A -> gasto médio: {np.mean(x_app_a):.2f}')
print(f'App - Grupo B -> gasto médio: {np.mean(x_app_b):.2f}\n')
gu.test_results(x_app_a, x_app_b, alpha=significance_lvl, verbose=True)

print(f'Test Lift: {test_lift(x_app_a.mean(), x_app_b.mean()):.2%}')

App - Grupo A -> gasto médio: 1888.41
App - Grupo B -> gasto médio: 1928.23

Mann Whitney U Test

P-value: 0.5713
Falha em rejeitar a hipótese nula.
Não há evidências suficientes no nível de significância 0.05 para dizer que existe diferença entre as médias  dos grupos.
Test Lift: -2.07%


## USA

In [18]:
dfs = gu.data(dataframes['USA'])

### Site

In [19]:
sample_n = data_sizes_countries_site['USA']
x_site_a = dfs['site_a'].sample(sample_n, random_state=0).spent
x_site_b = dfs['site_b'].sample(sample_n, random_state=0).spent

print(f'Site - Grupo A -> gasto médio: {np.mean(x_site_a):.2f}')
print(f'Site - Grupo B -> gasto médio: {np.mean(x_site_b):.2f}\n')
gu.test_results(x_site_a, x_site_b, alpha=significance_lvl, verbose=True)

print(f'Test Lift: {test_lift(x_site_a.mean(), x_site_b.mean()):.2%}')

Site - Grupo A -> gasto médio: 1919.00
Site - Grupo B -> gasto médio: 1882.38

Mann Whitney U Test

P-value: 0.4881
Falha em rejeitar a hipótese nula.
Não há evidências suficientes no nível de significância 0.05 para dizer que existe diferença entre as médias  dos grupos.
Test Lift: 1.95%


### App

In [20]:
sample_n = data_sizes_countries_app['USA']
x_app_a = dfs['app_a'].sample(sample_n, random_state=0).spent
x_app_b = dfs['app_b'].sample(sample_n, random_state=0).spent
print(f'App - Grupo A -> gasto médio: {np.mean(x_app_a):.2f}')
print(f'App - Grupo B -> gasto médio: {np.mean(x_app_b):.2f}\n')
gu.test_results(x_app_a, x_app_b, alpha=significance_lvl, verbose=True)

print(f'Test Lift: {test_lift(x_app_a.mean(), x_app_b.mean()):.2%}')

App - Grupo A -> gasto médio: 1888.14
App - Grupo B -> gasto médio: 1858.35

Mann Whitney U Test

P-value: 0.2004
Falha em rejeitar a hipótese nula.
Não há evidências suficientes no nível de significância 0.05 para dizer que existe diferença entre as médias  dos grupos.
Test Lift: 1.60%


## Brazil

### Site

In [21]:
dfs = gu.data(dataframes['BRA'])

In [22]:
sample_n = data_sizes_countries_site['BRA']
x_site_a = dfs['site_a'].sample(sample_n, random_state=0).spent
x_site_b = dfs['site_b'].sample(sample_n, random_state=0).spent

print(f'Site - Grupo A -> gasto médio: {np.mean(x_site_a):.2f}')
print(f'Site - Grupo B -> gasto médio: {np.mean(x_site_b):.2f}\n')
gu.test_results(x_site_a, x_site_b, alpha=significance_lvl, verbose=True)

print(f'Test Lift: {test_lift(x_site_a.mean(), x_site_b.mean()):.2%}')

Site - Grupo A -> gasto médio: 1915.17
Site - Grupo B -> gasto médio: 1933.36

Mann Whitney U Test

P-value: 0.5135
Falha em rejeitar a hipótese nula.
Não há evidências suficientes no nível de significância 0.05 para dizer que existe diferença entre as médias  dos grupos.
Test Lift: -0.94%


### App

In [23]:
sample_n = data_sizes_countries_app['BRA']
x_app_a = dfs['app_a'].sample(sample_n, random_state=0).spent
x_app_b = dfs['app_b'].sample(sample_n, random_state=0).spent

print(f'App - Grupo A -> gasto médio: {np.mean(x_app_a):.2f}')
print(f'App - Grupo B -> gasto médio: {np.mean(x_app_b):.2f}\n')
gu.test_results(x_app_a, x_app_b, alpha=significance_lvl, verbose=True)


print(f'Test Lift: {test_lift(x_app_a.mean(), x_app_b.mean()):.2%}')

App - Grupo A -> gasto médio: 1878.62
App - Grupo B -> gasto médio: 1920.99

Mann Whitney U Test

P-value: 0.4059
Falha em rejeitar a hipótese nula.
Não há evidências suficientes no nível de significância 0.05 para dizer que existe diferença entre as médias  dos grupos.
Test Lift: -2.21%


## 4.3 Resumo

In [24]:
countries_test = ['MEX', 'USA', 'BRA']

results_site, results_app = gu.tests(countries_test, metric='spent',dataframes=dataframes, sizes_dict_site=data_sizes_countries_site, sizes_dict_app=data_sizes_countries_app)

In [25]:
results_site.T

,País,Dispositivo,Teste,Hipótese,p-value,Comentário
0,MEX,Site,Mann Whitney U Test,Falha em rejeitar a hipótese nula.,0.120359,Não há evidências suficientes no nível de significância 0.05 para dizer que existe diferença entre as médias dos grupos.
0,USA,Site,Mann Whitney U Test,Falha em rejeitar a hipótese nula.,0.488118,Não há evidências suficientes no nível de significância 0.05 para dizer que existe diferença entre as médias dos grupos.
0,BRA,Site,Mann Whitney U Test,Falha em rejeitar a hipótese nula.,0.513473,Não há evidências suficientes no nível de significância 0.05 para dizer que existe diferença entre as médias dos grupos.


In [26]:
results_app.T

,País,Dispositivo,Teste,Hipótese,p-value,Comentário
0,MEX,App,Mann Whitney U Test,Falha em rejeitar a hipótese nula.,0.571257,Não há evidências suficientes no nível de significância 0.05 para dizer que existe diferença entre as médias dos grupos.
0,USA,App,Mann Whitney U Test,Falha em rejeitar a hipótese nula.,0.200415,Não há evidências suficientes no nível de significância 0.05 para dizer que existe diferença entre as médias dos grupos.
0,BRA,App,Mann Whitney U Test,Falha em rejeitar a hipótese nula.,0.405948,Não há evidências suficientes no nível de significância 0.05 para dizer que existe diferença entre as médias dos grupos.


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)


# 5.0 Resultados

Com os parâmetros utilizados, não foi possível realizar testes para os seguintes países: Espanha, Grã Bretanha, Turquia, Alemanha, França, Austria e Canada. Então reportamos ao time de negócio que precisamos coletar mais dados para estes países, se quisermos realizar testes com resultados confiáveis.

Foi possível realizar os testes para Estados Unidos, Brasil e México — tanto para o site quanto para o app. Mas não obtivemos resultados estatisticamente significativos para dizer que há diferença entre as médias do preenchimento automático e manual.

Possíveis cenários:

- Existe algum problema no formulário de preenchimento automático que esteja atrapalhando a sua performance. E devemos realizar novos testes após a correção do problema.
- Não há problema com o formulário de preenchimento automático. Este método apenas não funciona para estes países e não vale a pena implementa-lo.
- Problema no processo de aleatorização dos elementos dos grupos.

OBS: É recomenado realizar vários testes A/A, antes e/ou em paralelo, ao teste A/B para descobrirmos se há algum problema em nosso exprimento. Por exemplo: 
- Anomalias na plataforma, 
- Verificar se o erro tipo I está controlado
- Variação da métrica ao longo do tempo
- Verificar se há algum viés entre tratamento e controle.

## 6.0 Referências

Kohavi, R., D. Tang, Y. Xu. Trustworthy Online Controlled Experiments: A Practical Guide to A/B Testing.
Cambridge, UK: Cambridge University Press, in press.

Stefan Thomke. Experimentation works : the surprising power of business experiments. Boston, Massachusetts : Harvard Business Review Press.

Cohen, J. Statistical Power Analysis for the Behavioral Sciences. Department of Psychology, New York University.

https://towardsdatascience.com/simple-and-complet-guide-to-a-b-testing-c34154d0ce5a